In [10]:
import json
import re

In [11]:
with open(r"C:\E old\DS310\dataset\legal_corpus.json", "r", encoding="utf-8") as f:
    law_corpus = json.load(f)

In [12]:
law_corpus[0]

{'id': 0,
 'law_id': '14/2022/TT-NHNN',
 'content': [{'aid': 0,
   'content_Article': '1. Thông tư này quy định mã số, tiêu chuẩn chuyên môn, nghiệp vụ và xếp lương đối với các ngạch công chức chuyên ngành Ngân hàng.\n\n2. Thông tư này áp dụng đối với công chức làm việc tại các đơn vị thuộc Ngân hàng Nhà nước Việt Nam (gọi tắt là Ngân hàng Nhà nước).'},
  {'aid': 1,
   'content_Article': '1. Kiểm soát viên cao cấp ngân hàng Mã số: 07.044 2. Kiểm soát viên chính ngân hàng Mã số: 07.045 3. Kiểm soát viên ngân hàng Mã số: 07.046 4. Thủ kho, thủ quỹ ngân hàng Mã số: 07.048 5. Nhân viên Tiền tệ - Kho quỹ Mã số: 07.047 '},
  {'aid': 2,
   'content_Article': '1. Có bản lĩnh chính trị vững vàng, kiên định với chủ nghĩa Mác - Lênin, tư tưởng Hồ Chí Minh; nắm vững chủ trương, đường lối, quy định của Đảng, chính sách, pháp luật của Nhà nước; trung thành với Tổ quốc; bảo vệ lợi ích của Nhà nước, của nhân dân.\n\n2. Thực hiện đầy đủ nghĩa vụ của công chức theo quy định của pháp luật; nghiêm túc chấ

## Corpus Merging

In [18]:
# Merge law_id base on the chars before last '/' example 14/2022/TT-NHNN & 11/2022/TT-NHNN -> merge to TT-NHNN
for i in range(len(law_corpus)):
    # print(law_corpus[i]['law_id'])
    law_id = law_corpus[i]['law_id']
    # Split law_id by '/'
    law_id_parts = law_id.split('/')
    # Get the last part
    last_part = law_id_parts[-1]
    # Merge law_id base on the last part
    law_corpus[i]['law_id'] = last_part

print(f"Trước khi merge: {len(law_corpus)} documents")

Trước khi merge: 2157 documents


In [19]:
# Gộp các documents có cùng law_id lại thành một document duy nhất
# Mỗi law_id sẽ có một document với tất cả articles được gộp lại

from collections import defaultdict

# Lưu thống kê trước khi merge (để so sánh)
before_merge_count = {}
for law in law_corpus:
    law_id = law['law_id']
    before_merge_count[law_id] = before_merge_count.get(law_id, 0) + 1

print(f"Trước khi merge: {len(law_corpus)} documents, {len(before_merge_count)} unique law_id")

# Dictionary để lưu articles theo law_id
law_dict = defaultdict(lambda: {'law_id': '', 'content': [], 'original_ids': []})

# Thu thập tất cả articles theo law_id
for law in law_corpus:
    law_id = law['law_id']
    law_dict[law_id]['law_id'] = law_id
    law_dict[law_id]['content'].extend(law['content'])  # Gộp tất cả articles
    law_dict[law_id]['original_ids'].append(law.get('id', None))

# Tạo corpus mới từ dictionary đã merge
merged_corpus = []
aid_counter = 0  # Để đảm bảo aid là unique

for idx, (law_id, law_data) in enumerate(law_dict.items()):
    # Đánh số lại aid để tránh trùng lặp
    content = []
    for article in law_data['content']:
        article_copy = article.copy()
        article_copy['aid'] = aid_counter
        aid_counter += 1
        content.append(article_copy)
    
    merged_law = {
        'id': idx,
        'law_id': law_id,
        'content': content,
        'num_articles': len(content),
        'original_ids': law_data['original_ids'],
        'merged_from': len(law_data['original_ids'])  # Số documents đã merge
    }
    merged_corpus.append(merged_law)

# Cập nhật law_corpus
law_corpus = merged_corpus

print(f"\nSau khi merge: {len(law_corpus)} documents")
print(f"Tổng số articles: {sum(len(law['content']) for law in law_corpus):,}")
print(f"Giảm {sum(before_merge_count.values()) - len(law_corpus)} documents")

print(f"\nVí dụ merged document:")
if len(law_corpus) > 0:
    sample = law_corpus[0]
    print(f"  Law ID: {sample['law_id']}")
    print(f"  Số articles: {sample['num_articles']}")
    print(f"  Đã merge từ: {sample.get('merged_from', 0)} documents")

Trước khi merge: 2157 documents, 125 unique law_id

Sau khi merge: 125 documents
Tổng số articles: 59,636
Giảm 2032 documents

Ví dụ merged document:
  Law ID: TT-NHNN
  Số articles: 1999
  Đã merge từ: 94 documents


In [20]:
# Thống kê số lượng articles cho mỗi law_id sau khi merge
dict_law_id = {}
for law in law_corpus:
    law_id = law['law_id']
    num_articles = len(law['content'])
    dict_law_id[law_id] = num_articles

print(f"Số lượng unique law_id: {len(dict_law_id)}")
print(f"\nTop 20 law_id có nhiều articles nhất:")
sorted_laws = sorted(dict_law_id.items(), key=lambda x: x[1], reverse=True)
for i, (law_id, count) in enumerate(sorted_laws[:20], 1):
    print(f"  {i:2d}. {law_id:<20s}: {count:>6,} articles")

Số lượng unique law_id: 125

Top 20 law_id có nhiều articles nhất:
   1. NĐ-CP               : 22,150 articles
   2. QH13                :  7,615 articles
   3. QH14                :  3,701 articles
   4. TT-BTC              :  2,731 articles
   5. QH12                :  2,131 articles
   6. TT-NHNN             :  1,999 articles
   7. TT-BGTVT            :  1,739 articles
   8. QH11                :  1,501 articles
   9. TT-BYT              :  1,309 articles
  10. TT-BCA              :  1,058 articles
  11. TT-BQP              :  1,053 articles
  12. QH15                :  1,038 articles
  13. TT-BTNMT            :    954 articles
  14. TT-BGDĐT            :    893 articles
  15. TT-BNNPTNT          :    872 articles
  16. TT-BCT              :    828 articles
  17. TT-BLĐTBXH          :    823 articles
  18. TT-BTP              :    678 articles
  19. QH10                :    612 articles
  20. QĐ-TTg              :    581 articles


In [23]:
len(law_corpus)

125

In [24]:
with open(r"C:\E old\DS310\dataset\legal_corpus_merged.json", "w", encoding="utf-8") as f:
    json.dump(law_corpus, f, ensure_ascii=False, indent=4)

## Train Merging

In [26]:
# Load train data
with open(r"C:\E old\DS310\dataset\train.json", "r", encoding="utf-8") as f:
    train_data = json.load(f)

print(f"Tổng số samples trong train: {len(train_data):,}")
print(f"\nVí dụ một example:")
print(train_data[0])

Tổng số samples trong train: 2,190

Ví dụ một example:
{'qid': 933, 'question': 'Thưa luật sư tôi có đăng ký kết hôn trên pháp luật nhưng nay vợ chồng bỏ nhau theo phong tục tập quán như vậy tôi có được phép kết hôn với người khác không ạ?', 'relevant_laws': [53877, 53875, 53929], 'answer': 'ở đây nếu vợ chồng bạn bỏ nhau nhưng chưa ly hôn (chưa có bản án/quyết định của Tòa) thì không thể kết hôn được với người khác. Do đó, chỉ khi vợ chồng bạn đã ly hôn theo bản án/quyết định của Tòa thì khi đó bạn mới có quyền kết hôn với người mới.'}


In [27]:
# Tạo mapping từ aid -> law_id
# Sau khi merge, mỗi article có aid unique và thuộc một law_id
aid_to_law_id = {}
for law in law_corpus:
    law_id = law['law_id']
    for article in law['content']:
        aid = article['aid']
        aid_to_law_id[aid] = law_id

print(f"Tổng số articles trong mapping: {len(aid_to_law_id):,}")
print(f"Ví dụ mapping:")
sample_aids = list(aid_to_law_id.keys())[:5]
for aid in sample_aids:
    print(f"  aid {aid} -> law_id: {aid_to_law_id[aid]}")

Tổng số articles trong mapping: 59,636
Ví dụ mapping:
  aid 0 -> law_id: TT-NHNN
  aid 1 -> law_id: TT-NHNN
  aid 2 -> law_id: TT-NHNN
  aid 3 -> law_id: TT-NHNN
  aid 4 -> law_id: TT-NHNN


In [28]:
# Tạo train_merged với cột law_id
train_merged = []
missing_aids = []  # Theo dõi các aid không tìm thấy trong corpus
missing_count = 0

for example in train_data:
    qid = example['qid']
    question = example['question']
    relevant_laws = example['relevant_laws']  # List các aid
    answer = example['answer']
    
    # Tìm law_id cho mỗi relevant_law (aid)
    law_ids = []
    for aid in relevant_laws:
        if aid in aid_to_law_id:
            law_id = aid_to_law_id[aid]
            if law_id not in law_ids:  # Tránh trùng lặp
                law_ids.append(law_id)
        else:
            missing_aids.append(aid)
            missing_count += 1
    
    # Tạo example mới với law_id
    merged_example = {
        'qid': qid,
        'question': question,
        'relevant_laws': relevant_laws,
        'law_id': law_ids,  # Thêm cột mới
        'answer': answer
    }
    train_merged.append(merged_example)

print(f"Đã xử lý: {len(train_merged):,} examples")
print(f"Số aid không tìm thấy trong corpus: {missing_count:,}")

if missing_count > 0:
    print(f"\nMột số aid không tìm thấy (sample):")
    unique_missing = list(set(missing_aids))[:10]
    for aid in unique_missing:
        print(f"  aid: {aid}")
    
print(f"\nVí dụ example sau khi merge:")
print(train_merged[0])

Đã xử lý: 2,190 examples
Số aid không tìm thấy trong corpus: 0

Ví dụ example sau khi merge:
{'qid': 933, 'question': 'Thưa luật sư tôi có đăng ký kết hôn trên pháp luật nhưng nay vợ chồng bỏ nhau theo phong tục tập quán như vậy tôi có được phép kết hôn với người khác không ạ?', 'relevant_laws': [53877, 53875, 53929], 'law_id': ['TT-BLĐTBXH'], 'answer': 'ở đây nếu vợ chồng bạn bỏ nhau nhưng chưa ly hôn (chưa có bản án/quyết định của Tòa) thì không thể kết hôn được với người khác. Do đó, chỉ khi vợ chồng bạn đã ly hôn theo bản án/quyết định của Tòa thì khi đó bạn mới có quyền kết hôn với người mới.'}


In [29]:
# Thống kê về law_id trong train_merged
from collections import Counter

# Đếm số lần mỗi law_id xuất hiện
law_id_counts = Counter()
examples_with_law_id = 0
examples_without_law_id = 0

for example in train_merged:
    law_ids = example['law_id']
    if len(law_ids) > 0:
        examples_with_law_id += 1
        for law_id in law_ids:
            law_id_counts[law_id] += 1
    else:
        examples_without_law_id += 1

print("THỐNG KÊ LAW_ID TRONG TRAIN_MERGED")
print("=" * 80)
print(f"Tổng số examples: {len(train_merged):,}")
print(f"Examples có law_id: {examples_with_law_id:,} ({examples_with_law_id/len(train_merged)*100:.2f}%)")
print(f"Examples không có law_id: {examples_without_law_id:,} ({examples_without_law_id/len(train_merged)*100:.2f}%)")
print(f"Số unique law_id: {len(law_id_counts):,}")

print(f"\n{'─' * 80}")
print("Top 20 law_id xuất hiện nhiều nhất trong train:")
for i, (law_id, count) in enumerate(law_id_counts.most_common(20), 1):
    print(f"  {i:2d}. {law_id:<20s}: {count:>6,} lần")

# Thống kê số law_id mỗi example
num_law_ids_per_example = [len(example['law_id']) for example in train_merged if len(example['law_id']) > 0]
if num_law_ids_per_example:
    print(f"\n{'─' * 80}")
    print("Thống kê số law_id mỗi example:")
    from collections import Counter
    law_id_per_example_dist = Counter(num_law_ids_per_example)
    for num_law_ids in sorted(law_id_per_example_dist.keys()):
        count = law_id_per_example_dist[num_law_ids]
        print(f"  {num_law_ids} law_id: {count:,} examples ({count/len(num_law_ids_per_example)*100:.2f}%)")


THỐNG KÊ LAW_ID TRONG TRAIN_MERGED
Tổng số examples: 2,190
Examples có law_id: 2,190 (100.00%)
Examples không có law_id: 0 (0.00%)
Số unique law_id: 81

────────────────────────────────────────────────────────────────────────────────
Top 20 law_id xuất hiện nhiều nhất trong train:
   1. NĐ-CP               :    769 lần
   2. QH13                :    235 lần
   3. QH14                :    105 lần
   4. TT-BGDĐT            :     97 lần
   5. QH12                :     96 lần
   6. TT-BLĐTBXH          :     89 lần
   7. TT-NHNN             :     87 lần
   8. TT-BCT              :     71 lần
   9. TT-BGTVT            :     63 lần
  10. TT-BYT              :     63 lần
  11. QH10                :     62 lần
  12. TT-BTC              :     58 lần
  13. TT-BTTTT            :     46 lần
  14. QH11                :     44 lần
  15. TT-BNNPTNT          :     40 lần
  16. QH15                :     36 lần
  17. TT-BTNMT            :     29 lần
  18. TT-BQP              :     26 lần
  19. TT-BKHĐT  

In [30]:
# Lưu train_merged ra file
output_path = r"C:\E old\DS310\dataset\train_merged.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(train_merged, f, ensure_ascii=False, indent=2)

print(f"✅ Đã lưu train_merged vào: {output_path}")
print(f"   Tổng số examples: {len(train_merged):,}")
print(f"   File size: {len(json.dumps(train_merged, ensure_ascii=False)) / (1024*1024):.2f} MB (estimated)")


✅ Đã lưu train_merged vào: C:\E old\DS310\dataset\train_merged.json
   Tổng số examples: 2,190
   File size: 2.40 MB (estimated)


In [31]:
# Tìm các law_id không có hoặc có ít trong train_merged
# So sánh với tất cả law_id có trong corpus

# Tất cả law_id trong corpus
all_law_ids_in_corpus = set([law['law_id'] for law in law_corpus])
print(f"Tổng số law_id trong corpus: {len(all_law_ids_in_corpus)}")

# Đếm số lần xuất hiện mỗi law_id trong train_merged
law_id_counts_in_train = Counter()
for example in train_merged:
    for law_id in example['law_id']:
        law_id_counts_in_train[law_id] += 1

print(f"Số law_id xuất hiện trong train_merged: {len(law_id_counts_in_train)}")
print(f"Số law_id KHÔNG xuất hiện trong train_merged: {len(all_law_ids_in_corpus - set(law_id_counts_in_train.keys()))}")

# Tạo dictionary với tất cả law_id (bao gồm cả những cái có 0 lần)
all_law_id_stats = {}
for law_id in all_law_ids_in_corpus:
    count = law_id_counts_in_train.get(law_id, 0)
    # Lấy số articles của law_id từ corpus để so sánh
    law_doc = next((law for law in law_corpus if law['law_id'] == law_id), None)
    num_articles = len(law_doc['content']) if law_doc else 0
    all_law_id_stats[law_id] = {
        'count_in_train': count,
        'num_articles': num_articles,
        'coverage_ratio': count / len(train_merged) * 100 if len(train_merged) > 0 else 0
    }

# Sắp xếp theo count_in_train tăng dần (ít nhất trước)
sorted_law_ids = sorted(all_law_id_stats.items(), key=lambda x: (x[1]['count_in_train'], -x[1]['num_articles']))

print("\n" + "=" * 100)
print("TOP 50 LAW_ID KHÔNG CÓ HOẶC CÓ ÍT NHẤT TRONG TRAIN_MERGED")
print("=" * 100)
print(f"{'STT':<5} {'Law ID':<25} {'Xuất hiện':<12} {'Số articles':<15} {'Coverage (%)':<15}")
print("-" * 100)

for i, (law_id, stats) in enumerate(sorted_law_ids[:50], 1):
    count = stats['count_in_train']
    num_articles = stats['num_articles']
    coverage = stats['coverage_ratio']
    status = "❌ KHÔNG CÓ" if count == 0 else f"⚠️  ÍT ({count} lần)"
    print(f"{i:<5} {law_id:<25} {status:<12} {num_articles:>10,}        {coverage:>6.3f}%")

# Thống kê tổng hợp
zero_count = sum(1 for stats in all_law_id_stats.values() if stats['count_in_train'] == 0)
low_count = sum(1 for stats in all_law_id_stats.values() if 0 < stats['count_in_train'] < 5)

print("\n" + "=" * 100)
print("THỐNG KÊ TỔNG HỢP")
print("=" * 100)
print(f"Law_id KHÔNG có trong train (0 lần): {zero_count} law_id")
print(f"Law_id có ÍT trong train (1-4 lần): {low_count} law_id")
print(f"Law_id có ĐỦ trong train (≥5 lần): {len(all_law_ids_in_corpus) - zero_count - low_count} law_id")

Tổng số law_id trong corpus: 125
Số law_id xuất hiện trong train_merged: 81
Số law_id KHÔNG xuất hiện trong train_merged: 44

TOP 50 LAW_ID KHÔNG CÓ HOẶC CÓ ÍT NHẤT TRONG TRAIN_MERGED
STT   Law ID                    Xuất hiện    Số articles     Coverage (%)   
----------------------------------------------------------------------------------------------------
1     HĐNN8                     ❌ KHÔNG CÓ           54         0.000%
2     PL-UBTVQH12               ❌ KHÔNG CÓ           25         0.000%
3     QĐ-BTTTT                  ❌ KHÔNG CÓ           25         0.000%
4     TTLT-BKHĐT-BTC            ❌ KHÔNG CÓ           24         0.000%
5     TT-BLDTBXH                ❌ KHÔNG CÓ           24         0.000%
6     TTLT-BTP-BCA-BYT          ❌ KHÔNG CÓ           21         0.000%
7     TTLT-BTTTT-BNV            ❌ KHÔNG CÓ           21         0.000%
8     TTLT-BYT-BNN&PTNT         ❌ KHÔNG CÓ           20         0.000%
9     TTLT-BYT-BCA              ❌ KHÔNG CÓ           20         0.000%

In [32]:
# Phân tích chi tiết hơn: xem những law_id nào không có trong train nhưng có nhiều articles
print("\n" + "=" * 100)
print("TOP 20 LAW_ID KHÔNG CÓ TRONG TRAIN NHƯNG CÓ NHIỀU ARTICLES (ĐÁNG CHÚ Ý)")
print("=" * 100)

missing_law_ids_with_articles = [
    (law_id, stats) 
    for law_id, stats in all_law_id_stats.items() 
    if stats['count_in_train'] == 0
]
missing_law_ids_with_articles.sort(key=lambda x: x[1]['num_articles'], reverse=True)

print(f"{'STT':<5} {'Law ID':<25} {'Số articles':<15} {'Đánh giá'}")
print("-" * 100)

for i, (law_id, stats) in enumerate(missing_law_ids_with_articles[:20], 1):
    num_articles = stats['num_articles']
    if num_articles >= 100:
        evaluation = "🔴 QUAN TRỌNG (≥100 articles)"
    elif num_articles >= 50:
        evaluation = "🟡 ĐÁNG CHÚ Ý (≥50 articles)"
    else:
        evaluation = "🟢 Bình thường"
    print(f"{i:<5} {law_id:<25} {num_articles:>10,}        {evaluation}")

print(f"\nTổng số law_id không có trong train: {len(missing_law_ids_with_articles)}")
if len(missing_law_ids_with_articles) > 0:
    total_articles_missing = sum(stats['num_articles'] for _, stats in missing_law_ids_with_articles)
    print(f"Tổng số articles của các law_id không có trong train: {total_articles_missing:,}")


TOP 20 LAW_ID KHÔNG CÓ TRONG TRAIN NHƯNG CÓ NHIỀU ARTICLES (ĐÁNG CHÚ Ý)
STT   Law ID                    Số articles     Đánh giá
----------------------------------------------------------------------------------------------------
1     HĐNN8                             54        🟡 ĐÁNG CHÚ Ý (≥50 articles)
2     PL-UBTVQH12                       25        🟢 Bình thường
3     QĐ-BTTTT                          25        🟢 Bình thường
4     TTLT-BKHĐT-BTC                    24        🟢 Bình thường
5     TT-BLDTBXH                        24        🟢 Bình thường
6     TTLT-BTP-BCA-BYT                  21        🟢 Bình thường
7     TTLT-BTTTT-BNV                    21        🟢 Bình thường
8     TTLT-BYT-BNN&PTNT                 20        🟢 Bình thường
9     TTLT-BYT-BCA                      20        🟢 Bình thường
10    TTLT-BKHCN-BTC                    18        🟢 Bình thường
11    TT-BCA(C13)                       16        🟢 Bình thường
12    TTLT-BTC-NHNNVN                   16        🟢

## Build Train for Classifier

In [39]:
# Xác định các law_id cần bổ sung vào train (không có hoặc < 5 lần)
underrepresented_law_ids = [
    law_id for law_id, stats in all_law_id_stats.items() 
    if stats['count_in_train'] < 5
]

print(f"Số law_id cần bổ sung (< 5 lần trong train): {len(underrepresented_law_ids)}")
print(f"\nDanh sách law_id cần bổ sung:")
for i, law_id in enumerate(underrepresented_law_ids, 1):
    count = all_law_id_stats[law_id]['count_in_train']
    num_articles = all_law_id_stats[law_id]['num_articles']
    print(f"  {i:2d}. {law_id:<25s} - Xuất hiện: {count:>2} lần, Articles: {num_articles:>5,}")

# Tính tổng số articles sẽ được thêm vào
total_articles_to_add = sum(all_law_id_stats[law_id]['num_articles'] for law_id in underrepresented_law_ids)
print(f"\nTổng số articles sẽ được thêm vào train: {total_articles_to_add:,}")

Số law_id cần bổ sung (< 5 lần trong train): 86

Danh sách law_id cần bổ sung:
   1. UBTVQH15                  - Xuất hiện:  1 lần, Articles:    36
   2. TTLT-BTC-BGTVT            - Xuất hiện:  4 lần, Articles:    17
   3. QĐ-HĐTĐQH                 - Xuất hiện:  0 lần, Articles:     3
   4. QĐ-TCHQ                   - Xuất hiện:  1 lần, Articles:    12
   5. QĐ-BCA-C06                - Xuất hiện:  0 lần, Articles:     3
   6. TTLT-BTC-BKHCN            - Xuất hiện:  0 lần, Articles:     7
   7. TTLT-BYT-BLĐTBXH          - Xuất hiện:  1 lần, Articles:     6
   8. PL-UBTVQH12               - Xuất hiện:  0 lần, Articles:    25
   9. QĐ-UBQGCĐS                - Xuất hiện:  0 lần, Articles:     3
  10. QĐ-TANDTC-TĐKT            - Xuất hiện:  0 lần, Articles:     3
  11. TTLN                      - Xuất hiện:  0 lần, Articles:     4
  12. QĐ-BKHCN                  - Xuất hiện:  0 lần, Articles:     3
  13. TTLT-BYT-BGDĐT            - Xuất hiện:  2 lần, Articles:    20
  14. TT-VPCP           

In [40]:
# Tách các examples có nhiều law_id thành nhiều samples
# Mỗi sample chỉ có 1 law_id, nhưng cùng câu hỏi
train_split = []
max_qid = max([ex['qid'] for ex in train_merged])
qid_counter = max_qid + 1  # Bắt đầu qid mới từ số lớn nhất + 1

for example in train_merged:
    law_ids = example['law_id']
    
    if len(law_ids) == 1:
        # Nếu chỉ có 1 law_id, giữ nguyên qid gốc
        train_split.append(example)
    else:
        # Nếu có nhiều law_id, tách thành nhiều samples với qid mới
        for law_id in law_ids:
            # Chỉ lấy relevant_laws thuộc law_id này
            relevant_aids_for_this_law = []
            for aid in example['relevant_laws']:
                if aid in aid_to_law_id and aid_to_law_id[aid] == law_id:
                    relevant_aids_for_this_law.append(aid)
            
            # Nếu có ít nhất 1 relevant_law thuộc law_id này
            if len(relevant_aids_for_this_law) > 0:
                split_example = {
                    'qid': qid_counter,
                    'question': example['question'],
                    'relevant_laws': relevant_aids_for_this_law,
                    'law_id': [law_id],  # Chỉ 1 law_id
                    'answer': example['answer']
                }
                train_split.append(split_example)
                qid_counter += 1

print(f"Số examples gốc: {len(train_merged):,}")
print(f"Số examples sau khi tách: {len(train_split):,}")
print(f"Tăng thêm: {len(train_split) - len(train_merged):,} examples")
print(f"QID bắt đầu cho samples mới: {max_qid + 1}")

# Kiểm tra phân bố law_id sau khi tách
law_id_counts_after_split = Counter()
for ex in train_split:
    for law_id in ex['law_id']:
        law_id_counts_after_split[law_id] += 1

print(f"\nSố unique law_id sau khi tách: {len(law_id_counts_after_split)}")
print(f"\nTop 10 law_id sau khi tách:")
for i, (law_id, count) in enumerate(law_id_counts_after_split.most_common(10), 1):
    print(f"  {i:2d}. {law_id:<20s}: {count:>6,} lần")

# Lưu qid_counter để dùng cho classifier_train_samples
print(f"\nQID tiếp theo sẽ dùng cho classifier samples: {qid_counter}")

Số examples gốc: 2,190
Số examples sau khi tách: 2,353
Tăng thêm: 163 examples
QID bắt đầu cho samples mới: 16758

Số unique law_id sau khi tách: 81

Top 10 law_id sau khi tách:
   1. NĐ-CP               :    769 lần
   2. QH13                :    235 lần
   3. QH14                :    105 lần
   4. TT-BGDĐT            :     97 lần
   5. QH12                :     96 lần
   6. TT-BLĐTBXH          :     89 lần
   7. TT-NHNN             :     87 lần
   8. TT-BCT              :     71 lần
   9. TT-BGTVT            :     63 lần
  10. TT-BYT              :     63 lần

QID tiếp theo sẽ dùng cho classifier samples: 17075


In [41]:
# Tạo training samples từ articles của các law_id thiếu
# Format: question = nội dung article, law_id = law_id của article đó, relevant_laws = [aid của article]
classifier_train_samples = []

for law_id in underrepresented_law_ids:
    # Tìm law document trong corpus
    law_doc = next((law for law in law_corpus if law['law_id'] == law_id), None)
    if law_doc is None:
        continue
    
    # Với mỗi article trong law_id này
    for article in law_doc['content']:
        aid = article['aid']
        article_content = article['content_Article']
        
        # Tạo sample cho classifier
        # Question: nội dung article (dùng toàn bộ)
        question_text = article_content.strip()
        
        # Answer: có thể để trống hoặc dùng nội dung article (tùy chọn)
        # Ở đây để trống vì question đã chứa đủ thông tin
        
        sample = {
            'qid': qid_counter,
            'question': question_text,  # Dùng toàn bộ nội dung article
            'relevant_laws': [aid],     # Chỉ có 1 aid - chính article này
            'law_id': [law_id],         # Chỉ có 1 law_id
            'answer': ''                # Để trống
        }
        classifier_train_samples.append(sample)
        qid_counter += 1

print(f"Đã tạo {len(classifier_train_samples):,} samples từ articles của {len(underrepresented_law_ids)} law_id thiếu")
print(f"\nVí dụ một sample:")
if len(classifier_train_samples) > 0:
    sample = classifier_train_samples[0]
    print(f"  QID: {sample['qid']}")
    print(f"  Law ID: {sample['law_id']}")
    print(f"  Question length: {len(sample['question'])} chars")
    print(f"  Question (first 200 chars): {sample['question'][:200]}...")
    print(f"  Relevant laws: {sample['relevant_laws']}")

# Thống kê theo law_id
law_id_article_counts = Counter()
for sample in classifier_train_samples:
    for law_id in sample['law_id']:
        law_id_article_counts[law_id] += 1

print(f"\nSố articles mỗi law_id được thêm vào:")
for law_id in underrepresented_law_ids[:10]:
    if law_id in law_id_article_counts:
        print(f"  {law_id:<25s}: {law_id_article_counts[law_id]:>6,} articles")

Đã tạo 1,986 samples từ articles của 86 law_id thiếu

Ví dụ một sample:
  QID: 17075
  Law ID: ['UBTVQH15']
  Question length: 21888 chars
  Question (first 200 chars): 1. Sửa đổi, bổ sung Điều 2 như sau:“Điều 2. Mục đích, nguyên tắc phân loại đô thị 1. Phân loại đô thị nhằm xác lập cơ sở đánh giá chất lượng đô thị; tổ chức, sắp xếp và quản lý, phát triển hệ thống đô...
  Relevant laws: [59078]

Số articles mỗi law_id được thêm vào:
  UBTVQH15                 :     36 articles
  TTLT-BTC-BGTVT           :     17 articles
  QĐ-HĐTĐQH                :      3 articles
  QĐ-TCHQ                  :     12 articles
  QĐ-BCA-C06               :      3 articles
  TTLT-BTC-BKHCN           :      7 articles
  TTLT-BYT-BLĐTBXH         :      6 articles
  PL-UBTVQH12              :     25 articles
  QĐ-UBQGCĐS               :      3 articles
  QĐ-TANDTC-TĐKT           :      3 articles


In [36]:
# Gộp train_split (đã tách) với classifier_train_samples (từ articles)
final_train_data = train_split + classifier_train_samples

print("THỐNG KÊ TẬP TRAIN CUỐI CÙNG")
print("=" * 100)
print(f"Tổng số samples: {len(final_train_data):,}")
print(f"  - Từ train gốc (đã tách): {len(train_split):,}")
print(f"  - Từ articles của law_id thiếu: {len(classifier_train_samples):,}")

# Thống kê law_id trong tập cuối cùng
final_law_id_counts = Counter()
for ex in final_train_data:
    for law_id in ex['law_id']:
        final_law_id_counts[law_id] += 1

print(f"\nSố unique law_id trong tập cuối: {len(final_law_id_counts)}")
print(f"\nPhân bố law_id:")
zero_count_final = sum(1 for count in final_law_id_counts.values() if count == 0)
low_count_final = sum(1 for count in final_law_id_counts.values() if 0 < count < 5)
sufficient_count_final = sum(1 for count in final_law_id_counts.values() if count >= 5)

print(f"  - Law_id KHÔNG có (0 lần): {zero_count_final}")
print(f"  - Law_id có ÍT (1-4 lần): {low_count_final}")
print(f"  - Law_id có ĐỦ (≥5 lần): {sufficient_count_final}")

# Kiểm tra xem tất cả law_id đã có đủ samples chưa
print(f"\n{'─' * 100}")
print("Kiểm tra coverage:")
all_final_law_ids = set(final_law_id_counts.keys())
still_missing = all_law_ids_in_corpus - all_final_law_ids
if len(still_missing) > 0:
    print(f"  ⚠️  Vẫn còn {len(still_missing)} law_id chưa có trong train: {list(still_missing)[:5]}...")
else:
    print(f"  ✅ Tất cả {len(all_law_ids_in_corpus)} law_id đều có trong train")

THỐNG KÊ TẬP TRAIN CUỐI CÙNG
Tổng số samples: 4,339
  - Từ train gốc (đã tách): 2,353
  - Từ articles của law_id thiếu: 1,986

Số unique law_id trong tập cuối: 125

Phân bố law_id:
  - Law_id KHÔNG có (0 lần): 0
  - Law_id có ÍT (1-4 lần): 15
  - Law_id có ĐỦ (≥5 lần): 110

────────────────────────────────────────────────────────────────────────────────────────────────────
Kiểm tra coverage:
  ✅ Tất cả 125 law_id đều có trong train


In [42]:
# Lưu tập train cuối cùng cho classifier
# Chỉ giữ lại: qid, question, và law_id
classifier_train_simplified = []
for example in final_train_data:
    simplified_example = {
        'qid': example['qid'],
        'question': example['question'],
        'law_id': example['law_id']
    }
    classifier_train_simplified.append(simplified_example)

classifier_train_path = r"C:\E old\DS310\dataset\train_classifier.json"
with open(classifier_train_path, "w", encoding="utf-8") as f:
    json.dump(classifier_train_simplified, f, ensure_ascii=False, indent=2)

print(f"✅ Đã lưu train_classifier vào: {classifier_train_path}")
print(f"   Tổng số samples: {len(classifier_train_simplified):,}")
print(f"   Chỉ giữ lại: qid, question, law_id")
print(f"   File size: {len(json.dumps(classifier_train_simplified, ensure_ascii=False)) / (1024*1024):.2f} MB (estimated)")

✅ Đã lưu train_classifier vào: C:\E old\DS310\dataset\train_classifier.json
   Tổng số samples: 4,339
   Chỉ giữ lại: qid, question, law_id
   File size: 2.14 MB (estimated)


In [44]:
# Kiểm tra các law_id có ít samples (đặc biệt là chỉ có 1 sample)
law_id_sample_counts = Counter()
for ex in classifier_train_simplified:
    for law_id in ex['law_id']:
        law_id_sample_counts[law_id] += 1

# Tìm các law_id có ít samples
single_sample_law_ids = [law_id for law_id, count in law_id_sample_counts.items() if count == 1]
low_sample_law_ids = [(law_id, count) for law_id, count in law_id_sample_counts.items() if 1 < count < 5]

print("KIỂM TRA PHÂN BỐ SAMPLES THEO LAW_ID")
print("=" * 100)
print(f"Tổng số unique law_id: {len(law_id_sample_counts)}")

print(f"\n{'─' * 100}")
print(f"Law_id chỉ có 1 sample: {len(single_sample_law_ids)}")
if len(single_sample_law_ids) > 0:
    print("\nDanh sách law_id chỉ có 1 sample:")
    for i, law_id in enumerate(single_sample_law_ids, 1):
        print(f"  {i:2d}. {law_id}")

if len(low_sample_law_ids) > 0:
    print(f"\n{'─' * 100}")
    print(f"Law_id có 2-4 samples: {len(low_sample_law_ids)}")
    print("\nDanh sách law_id có 2-4 samples:")
    for i, (law_id, count) in enumerate(sorted(low_sample_law_ids, key=lambda x: x[1]), 1):
        print(f"  {i:2d}. {law_id:<25s}: {count} samples")

# Thống kê tổng hợp
print(f"\n{'─' * 100}")
print("PHÂN BỐ TỔNG HỢP:")
print(f"  - Law_id có 1 sample:   {len(single_sample_law_ids):>3} law_id")
print(f"  - Law_id có 2-4 samples: {len(low_sample_law_ids):>3} law_id")
print(f"  - Law_id có 5-10 samples: {sum(1 for c in law_id_sample_counts.values() if 5 <= c <= 10):>3} law_id")
print(f"  - Law_id có >10 samples: {sum(1 for c in law_id_sample_counts.values() if c > 10):>3} law_id")


KIỂM TRA PHÂN BỐ SAMPLES THEO LAW_ID
Tổng số unique law_id: 125

────────────────────────────────────────────────────────────────────────────────────────────────────
Law_id chỉ có 1 sample: 1

Danh sách law_id chỉ có 1 sample:
   1. QĐ-TTG

────────────────────────────────────────────────────────────────────────────────────────────────────
Law_id có 2-4 samples: 14

Danh sách law_id có 2-4 samples:
   1. QĐ-TLĐ                   : 2 samples
   2. QĐ-HĐTĐQH                : 3 samples
   3. QĐ-BCA-C06               : 3 samples
   4. QĐ-UBQGCĐS               : 3 samples
   5. QĐ-TANDTC-TĐKT           : 3 samples
   6. QĐ-BKHCN                 : 3 samples
   7. UBKTTW                   : 3 samples
   8. QĐ-CTN                   : 3 samples
   9. QĐ-BNN-KTHT              : 3 samples
  10. QĐ-TTCP                  : 3 samples
  11. QĐ-KBNN                  : 3 samples
  12. TTLT-BGDĐT-BNV-BTC       : 3 samples
  13. TTLN                     : 4 samples
  14. 01-TTLN                  : 4 samp

In [43]:
# So sánh trước và sau khi bổ sung
print("SO SÁNH TRƯỚC VÀ SAU KHI BỔ SUNG")
print("=" * 100)

print(f"\n{'Metric':<40} {'Trước':<20} {'Sau':<20} {'Thay đổi':<20}")
print("-" * 100)

before_total = len(train_merged)
after_total = len(final_train_data)
print(f"{'Tổng số samples':<40} {before_total:>15,}    {after_total:>15,}    {after_total - before_total:>15,}")

before_unique_law_ids = len(set(law_id for ex in train_merged for law_id in ex['law_id']))
after_unique_law_ids = len(final_law_id_counts)
print(f"{'Số unique law_id':<40} {before_unique_law_ids:>15}    {after_unique_law_ids:>15}    {after_unique_law_ids - before_unique_law_ids:>15}")

before_missing = len([law_id for law_id in all_law_ids_in_corpus if law_id_counts_in_train.get(law_id, 0) == 0])
after_missing = len(still_missing) if 'still_missing' in locals() else 0
print(f"{'Law_id KHÔNG có (0 lần)':<40} {before_missing:>15}    {after_missing:>15}    {after_missing - before_missing:>15}")

before_low = len([law_id for law_id in all_law_ids_in_corpus if 0 < law_id_counts_in_train.get(law_id, 0) < 5])
after_low = low_count_final
print(f"{'Law_id có ÍT (1-4 lần)':<40} {before_low:>15}    {after_low:>15}    {after_low - before_low:>15}")

before_sufficient = len([law_id for law_id in all_law_ids_in_corpus if law_id_counts_in_train.get(law_id, 0) >= 5])
after_sufficient = sufficient_count_final
print(f"{'Law_id có ĐỦ (≥5 lần)':<40} {before_sufficient:>15}    {after_sufficient:>15}    {after_sufficient - before_sufficient:>15}")

SO SÁNH TRƯỚC VÀ SAU KHI BỔ SUNG

Metric                                   Trước                Sau                  Thay đổi            
----------------------------------------------------------------------------------------------------
Tổng số samples                                    2,190              4,339              2,149
Số unique law_id                                      81                125                 44
Law_id KHÔNG có (0 lần)                               44                  0                -44
Law_id có ÍT (1-4 lần)                                42                 15                -27
Law_id có ĐỦ (≥5 lần)                                 39                110                 71
